In [1]:
# imports
import importlib as imp
import geopandas as gpd
import pandas as pd
import functions as f
# debug
imp.reload(f)
import streamlit as st

<module 'functions' from 'd:\\ITU\\Geospatial Data Science\\PROJECT\\functions.py'>

In [3]:
st.secrets["OPENROUTSERVICE_API"]

'5b3ce3597851110001cf62488e1d38b412854808ac6fe5f922a3c5bb'

In [23]:
# check which forest area is along the way
forest_areas_with_bikelanes = gpd.read_parquet('dataset/processed/forest_areas_with_bikelanes_wgs84_TEMP.parquet')
# Step 1: Ensure both GeoDataFrames are in the same CRS
if forest_areas_with_bikelanes.crs != f.DENMARK_CRS:
    forest_areas_with_bikelanes = forest_areas_with_bikelanes.to_crs(f.DENMARK_CRS)

### Get a route

In [48]:
import openrouteservice

# Replace 'your_api_key_here' with your actual OpenRouteService API key
client = openrouteservice.Client(key=st.secrets["OPENROUTSERVICE_API"])

# Define your coordinates
coordinates1 = [[11.581726, 55.606109], [11.699829, 55.592143]]  # Sjeland
coordinates4 = [[10.217285, 57.421294], [10.568847, 55.235288]] # super funky long, it will fail (no path)
coordinates5 = [[10.217285, 57.421294], [9.074707, 54.933455]]
coordinates6 = [[10.217285, 57.421294], [10.568847, 55.235288], [9.074707, 54.933455]]
coordinates2 = [[12.473907, 55.717571], [12.561798, 55.695132]] # in Copenhagen

coordinates = coordinates1

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=coordinates,
                           profile='cycling-regular',
                           format='geojson')

# The response 'routes' will be a GeoJSON object containing the route
gdf_routes = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes.crs = 'EPSG:4326'
gdf_routes.explore()

### We have to know which areas are checked already

In [25]:
gdf_denmark = gdf_routes.to_crs(f.DENMARK_CRS)
forest_areas_already_touced = gpd.overlay(forest_areas_with_bikelanes, gdf_denmark, how='intersection')

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_28568\2970590642.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 2 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  forest_areas_already_touced = gpd.overlay(forest_areas_with_bikelanes, gdf_denmark, how='intersection')


In [26]:
forest_areas_already_touced

,transfers,fare,segments,way_points,summary,geometry


### Buffer the route to be able to find the forest areas

whcihc can be reached by a small detour

In [27]:
gdf_denmark_50 = f.buffer_and_union_and_simplify_geopandas(gdf_denmark, 50)
gdf_denmark_100 = f.buffer_and_union_and_simplify_geopandas(gdf_denmark, 100)
gdf_denmark_1000 = f.buffer_and_union_and_simplify_geopandas(gdf_denmark, 1000)

buffer objects start
unary union starts
unary union ends
buffer objects start
unary union starts
unary union ends
buffer objects start
unary union starts
unary union ends


In [52]:
gdf_denmark_50.explore()

In [33]:
touched_forests = gpd.overlay(forest_areas_with_bikelanes, gdf_denmark_1000, how='intersection')

In [51]:
touched_forests["area"] = touched_forests.area
touched_forests = touched_forests.sort_values(by='area', ascending=False)
TOP_N = 2
print(touched_forests.to_crs(f.CRS).head(TOP_N).geometry.centroid)
touched_forests.head(TOP_N).explore()


12    POINT (11.68209 55.60313)
0     POINT (11.61442 55.60587)
dtype: geometry


C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_28568\456067777.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print(touched_forests.to_crs(f.CRS).head(TOP_N).geometry.centroid)


In [50]:
touched_forests.explore()

In [55]:
coordinates = [
    [11.581726, 55.606109],  # Original start point
    [11.699829, 55.592143],  # Original end point
    [11.68209, 55.60313],    # Additional point
    [11.61442, 55.60587],    # Additional point
    [11.57688, 55.61297],    # Additional point
    [11.61690, 55.60043],    # Additional point
    [11.62806, 55.60910]     # Additional point
]

coordinates = [
    [11.581726, 55.606109],  # Original start point
    [11.699829, 55.592143],  # Original end point
    [11.68209, 55.60313],    # Additional point,    # Additional point
    [11.57688, 55.61297],    # Additional point
    [11.61690, 55.60043],    # Additional point
    [11.62806, 55.60910]     # Additional point
]

coordinates = [
    [11.581726, 55.606109],  # Original start point
    [11.699829, 55.592143],  # Original end point
    [11.68209, 55.60313],    # Additional point
    [11.61442, 55.60587],    # Additional point
    [11.57688, 55.61297],    # Additional point
    [11.61690, 55.60043],    # Additional point
    [11.62806, 55.60910]     # Additional point
]

print(coordinates)

radiuses = [1000] * len(coordinates)  # 1000 meters for each point

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=coordinates,
                           profile='cycling-regular',
                           format='geojson',
                           radiuses=radiuses,
                           optimize_waypoints=True)

gdf_routes = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes.crs = 'EPSG:4326'
gdf_routes.explore()

[[11.581726, 55.606109], [11.699829, 55.592143], [11.68209, 55.60313], [11.61442, 55.60587], [11.57688, 55.61297], [11.6169, 55.60043], [11.62806, 55.6091]]


In [38]:
# order dataframe by area


,geometry,area
12,"POLYGON ((668339.239 6165079.848, 668301.084 6...",661055.688138
0,"POLYGON ((664431.353 6164998.727, 664388.994 6...",586086.185607
13,"POLYGON ((662791.525 6166271.966, 662891.192 6...",307482.938071
11,"POLYGON ((665174.376 6164834.719, 665156.992 6...",261281.925078
3,"POLYGON ((665822.575 6165773.013, 665818.828 6...",246963.305850
8,"POLYGON ((665408.762 6164249.303, 665424.525 6...",96766.298642
9,"POLYGON ((663327.827 6165918.931, 663330.928 6...",75707.187159
10,"MULTIPOLYGON (((664602.799 6165966.937, 664600...",68029.188812
4,"POLYGON ((668384.427 6164316.293, 668399.865 6...",59271.682936
2,"POLYGON ((671600.937 6163874.231, 671576.807 6...",35898.211838


In [34]:
# check which forest area is along the way
touched_forests.explore()

In [ ]:
# pick the coordinate

In [ ]:
# re-plan